# UNB - PPGI
# Processamento de Imagem
## Trabalho 07

Henrique Brandão

___

Dada a seguinte imagem 

[warning-triangle.jpg] 

Use processamento de imagens para gerar uma imagem binária da forma geométrica predominante e processamento morfológico para transformar em um objeto sólido.  

Usando o método dos momentos invariantes descubra qual das imagens abaixo mais se encaixa na imagem binária resultante do seu processamento.   

[ imagens.png]  

Entrega é um documento em PDF em formato de relatório. Código só em sessão de anexo/apêndice. Não precisa enrolar nem colocar background, mas precisa discutir o que você está fazendo e discutir os resultados. 

In [1]:
import cv2

import numpy as np
import seaborn as sns

from math import sqrt
from random import choice

from PIL import Image
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
f1, f2 = 'img01.jpg', 'img02.jpg'

x1, x2 = cv2.imread(f1), cv2.imread(f2)

In [4]:
def equalizacao(imagem):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    imagem = cv2.equalizeHist(imagem)
    return imagem